In [148]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import cross_validation # .20버전에서는 제외될 예정
import pandas as pd

#데이터 표준화

xy = np.loadtxt('data/white.csv',skiprows=1, delimiter=',',dtype=np.float32)
x_data = xy[:, 0:-1]; y_data = xy[:, [-1]]
y_binary = (y_data > 5)

#데이터 표준화

def minmaxscaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / denominator
    # noise term prevents the zero division: (denominator + 1e-7)

x_data = minmaxscaler(x_data)
y_data = pd.DataFrame(y_binary)
y_data = y_data.astype('category')
y_data = pd.get_dummies(y_data)
y_data = y_data.values # one_hot으로 만들기

#train, test set 나누기 
X_train, X_test, y_train, y_test = train_test_split(x_data,y_data, test_size=0.3 , random_state=777)

learning_rate = 0.001 #경사하강정도 지정

tf.reset_default_graph()
# Logistic regression
## Build graph using tensorflow operations
X = tf.placeholder(tf.float32, [None, 11]) # 학습데이터는 11개의 features
Y = tf.placeholder(tf.float32, [None, 2])

keep_prob = tf.placeholder(tf.float32) # dropout rate

W1 = tf.get_variable("W1", shape=[11, 44], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([44])) # bias는 가중치의 열의 개수에 맞춘다.
L1 = tf.nn.relu(tf.matmul(X,W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.get_variable("W2", shape=[44,22], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([22])) # bias는 가중치의 열의 개수에 맞춘다.
L2 = tf.nn.relu(tf.matmul(L1,W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[22,7], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([7])) # bias는 가중치의 열의 개수에 맞춘다.
L3 = tf.nn.relu(tf.matmul(L2,W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

W4 = tf.get_variable("W4", shape=[7,2], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([2])) # bias는 가중치의 열의 개수에 맞춘다.

Last_L = tf.matmul(L3, W4) + b4 # 최종 그래프

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Last_L, labels=Y))

# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam 짱....

#학습 : for문으로!!
with tf.Session() as sess:
    avg = 0
    tf.global_variables_initializer().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    for i in range(500):
        feed_dict = {X:X_train, Y:y_train, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        print('Epoch: %04d' % (i + 1), 'cost =', '{:.4f}'.format(c)) # 단계별 cost 정하기
        avg = avg + c
    avg = avg/500
    print('avg cost =', '{:.4f}'.format(avg))
    #정확도 계산
    correct = tf.equal(tf.argmax(Last_L, 1), tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
    print('acc : ', sess.run(accuracy, feed_dict={X: X_test, Y: y_test, keep_prob: 1}))